## Etapas

1. Criação de features
2. Tratamento de outliers
3. Normalização dos dados
4. Ajustes nos dados
5. Pipeline
6. Separação em treino/teste
7. Treinando modelo

In [ ]:
# @title Importação de bibliotecas

# Dataset
import pandas as pd
import numpy as np
import requests
from pathlib import Path

# Criação de features
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# Tratamento de outliers
from sklearn.base import BaseEstimator, TransformerMixin

# Ajustes nos dados
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Treinamento do modelo 
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, TimeSeriesSplit

In [ ]:
# @title Carregamento do dataset

github_link = "https://github.com/mfigueireddo/ciencia-de-dados/blob/ba579573c5b8a9246ca04f7da29bc2c74c8b362c/datasets/pre-pipeline_wildfires.parquet"
url = github_link.replace("/blob/", "/raw/")

local_file_path = Path("/content/raw_wildfires.parquet")

# Faz uma requisição HTTP GET ao GitHub
with requests.get(url, stream=True) as request:

    request.raise_for_status() # Confere se houve êxito

    with open(local_file_path , "wb") as file:
        for chunk in request.iter_content(chunk_size=1024*1024):
            if chunk:
                file.write(chunk)

wildfires = pd.read_parquet(local_file_path,  engine="pyarrow") # Leitura realizada com a engine pyarrow

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/QuickAccess/pre-pipeline_wildfires.csv'

## 1. Criação de features

Features geradas
- Estação do ano baseada no hemisfério Sul (0=Verão, 1=Outono, 2=Inverno, 3=Primavera)
- Região do incêndio
- Soma da precipitação nos últimos 14 dias
- Média de temperatura máxima nos últimos 7 dias

In [ ]:
# @title Classe responsável pela criação de features

class FeaturesCreation(BaseEstimator, TransformerMixin):
    
    def __init__(self,
                 # Colunas utilizadas
                 date_column='data', group_column='fire_id', 
                 latitude_column='latitude', longitude_column='longitude', 
                 precipitation_column='precipitacao', max_temperature_column='temp_max',

                 # Parâmetros personalizados para criação das features
                 precipitation_window_days=14, max_temperature_window_days=7,

                 # Parâmetros do DBSCAN
                 max_radius_km=1.0, min_samples=5):
        
        self.date_column = date_column
        self.group_column = group_column
        self.latitude_column = latitude_column
        self.longitude_column = longitude_column
        self.precipitation_column = precipitation_column
        self.max_temperature_column = max_temperature_column
        self.precipitation_window_days = precipitation_window_days
        self.max_temperature_window_days = max_temperature_window_days
        self.max_radius_km = max_radius_km
        self.min_samples = min_samples

        # Objetos aprendidos no fit
        self._dbscan_ = None
        self._nearest_neighbors_core_ = None
        self._core_labels_ = None
        self._max_radius_ = None

    # Conversão necessária para o DBSCAN
    @staticmethod
    def _to_radians(latitude_or_longitude):
        return np.radians(latitude_or_longitude.astype(float))
    
    @staticmethod
    def _month_to_season_south(month):
        # Estações (aprox.) para hemisfério sul:
        # Verão:     Dez(12)-Fev(2) -> 0
        # Outono:    Mar(3)-Mai(5)  -> 1
        # Inverno:   Jun(6)-Ago(8)  -> 2
        # Primavera: Set(9)-Nov(11) -> 3
        if month in (12, 1, 2):   return 0
        if month in (3, 4, 5):    return 1
        if month in (6, 7, 8):    return 2
        return 3  # 9,10,11

    def fit(self, dataframe, y=None):

        dataframe = dataframe.copy()

        # Desativa o DBSCAN caso não haja latitude e longitude
        missing_cols = [c for c in [self.latitude_column, self.longitude_column] if c not in dataframe.columns]
        if missing_cols:
            self._dbscan_ = None
            self._nearest_neighbors_core_ = None
            self._core_labels_ = None
            self._max_radius_ = None
            return self

        # Latitude/Longitude -> Radianos
        latitude_or_longitude = dataframe[[self.latitude_column, self.longitude_column]].to_numpy()
        latitude_or_longitude_radians = self._to_radians(latitude_or_longitude)

        # Raio/EPS (km -> radianos)
        earth_radius = 6371.0
        self._max_radius_ = self.max_radius_km / earth_radius

        # Executa o DBSCAN
        dbscan = DBSCAN(eps=self._max_radius_, min_samples=self.min_samples, metric='haversine')
        dbscan.fit(latitude_or_longitude_radians)
        self._dbscan_ = dbscan

        # Treina um NearestNeighbors apenas nos pontos-core para atribuição de novos pontos à clusters existentes em transform()
        core_mask = np.zeros_like(dbscan.labels_, dtype=bool)
        if hasattr(dbscan, 'core_sample_indices_') and len(dbscan.core_sample_indices_) > 0:
            core_mask[dbscan.core_sample_indices_] = True
            core_points = latitude_or_longitude_radians[core_mask]
            core_labels = dbscan.labels_[core_mask]

            if len(core_points) > 0:
                nearest_neighbors = NearestNeighbors(n_neighbors=1, metric='haversine')
                nearest_neighbors.fit(core_points)
                self._nearest_neighbors_core_ = nearest_neighbors
                self._core_labels_ = core_labels
            else:
                self._nearest_neighbors_core_ = None
                self._core_labels_ = None
        else:
            self._nearest_neighbors_core_ = None
            self._core_labels_ = None

        return self

    # Rotula novos pontos
    def _assign_dbscan_labels(self, dataframe):

        # Se não tivemos lat/lon ou DBSCAN treinado, devolve NaN
        if self._nearest_neighbors_core_ is None or self._core_labels_ is None or self._max_radius_ is None:
            return pd.Series([-1] * len(dataframe), index=dataframe.index, dtype='int64')

        # Latitude/Longitude -> radiano
        latitude_or_longitude = dataframe[[self.latitude_column, self.longitude_column]].to_numpy()
        latitude_or_longitude_radians = self._to_radians(latitude_or_longitude)

        # Atribui rótulo do core mais próximo, desde que dentro do Raio/EPS
        distances, indices = self._nearest_neighbors_core_.kneighbors(latitude_or_longitude_radians, n_neighbors=1, return_distance=True)
        distances = distances.reshape(-1)
        indices = indices.reshape(-1)

        labels = np.full(len(dataframe), -1, dtype='int64')
        within = distances <= self._max_radius_
        labels[within] = self._core_labels_[indices[within]]

        return pd.Series(labels, index=dataframe.index, dtype='int64')

    # Adiciona médias móveis e soma pro grupo de incêndio
    def _add_temporal_rollings(self, dataframe):
        
        # Ordena por grupo e tempo para garantir rolling correto
        if self.group_column in dataframe.columns and self.date_column in dataframe.columns:
            dataframe = dataframe.sort_values([self.group_column, self.date_column])
        else:
            # Se faltar algo, só ordena por data se existir
            if self.date_column in dataframe.columns:
                dataframe = dataframe.sort_values(self.date_column)

        # Rolling de precipitação (soma 14d)
        if self.precipitation_column in dataframe.columns:
            dataframe['soma_precipitacao_14d'] = (
                dataframe.groupby(self.group_column, dropna=False)[self.precipitation_column]
                  .rolling(self.precipitation_window_days, min_periods=1)
                  .sum()
                  .reset_index(level=0, drop=True)
            )
        else:
            dataframe['soma_precipitacao_14d'] = np.nan

        # Rolling de temp máxima (média 7d)
        if self.max_temperature_column in dataframe.columns:
            dataframe['media_temp_max_7d'] = (
                dataframe.groupby(self.group_column, dropna=False)[self.max_temperature_column]
                  .rolling(self.max_temperature_window_days, min_periods=1)
                  .mean()
                  .reset_index(level=0, drop=True)
            )
        else:
            dataframe['media_temp_max_7d'] = np.nan

        return dataframe

    # Aplica transformações e cria as novas features
    def transform(self, dataframe, target=None):
        
        # Trabalha em DataFrame para manter nomes/índices
        dataframe = pd.DataFrame(dataframe).copy()

        # 1) Estação do ano (numérica, 0..3)
        if self.date_column in dataframe.columns:
            # Garante dtype datetime
            dataframe[self.date_column] = pd.to_datetime(dataframe[self.date_column], errors='coerce')
            estacao = dataframe[self.date_column].dt.month.map(self._month_to_season_south).astype('Int64')
            dataframe['estacao_ano_id'] = estacao.astype('float').astype('Int64')  # evita problemas de NaN -> imputar depois
            dataframe['estacao_ano_id'] = dataframe['estacao_ano_id'].astype('float')
        else:
            dataframe['estacao_ano_id'] = np.nan

        # 2) Região por DBSCAN (atribuída por NN para dados novos)
        if all(c in dataframe.columns for c in [self.latitude_column, self.longitude_column]):
            dataframe['regiao_incendio'] = self._assign_dbscan_labels(dataframe).astype('int64')
        else:
            dataframe['regiao_incendio'] = -1

        # 3) Rollings temporais por grupo
        dataframe = self._add_temporal_rollings(dataframe)

        return dataframe


### Ajustes nos dados

As features estação do ano e região do incêndio que foram criadas precisam ser tratadas. 
Isso porque seus valores são categóricos e não há relação numérica entre eles. 
Isto será feito na normalização dos dados

## 2. Tratamento de outliers

**Transformação Logarítmica**
- Aplica log(x) ou log(x+constante) para valores positivos
- Muito eficaz para dados com distribuição assimétrica positiva
- Comprime valores grandes e expande valores pequenos
- Fórmula: X_log = log(X + c), onde c evita log(0)

**Transformação Raiz Quadrada**
- Menos drástica que a transformação logarítmica
- Útil para dados de contagem e variáveis positivamente assimétricas
- Fórmula: X_sqrt = sqrt(X)

**Winsorização (Capping/Clipping)**

A **Winsorização** é uma técnica de tratamento de outliers que **limita valores extremos** sem removê-los completamente. Em vez de excluir outliers, substituímos os valores extremos pelos valores de percentis específicos.

**Como funciona:**
- Define-se limites baseados em percentis (ex: 5º e 95º percentil)
- Valores abaixo do limite inferior são substituídos pelo valor do limite inferior
- Valores acima do limite superior são substituídos pelo valor do limite superior

| Variável                                 | Melhor método         | Justificativa (1 linha)                                                                                   |
| :--------------------------------------- | :-------------------- | :-------------------------------------------------------------------------------------------------------- |
| **precipitacao**                         | **Log(x + 1)**        | Reduziu fortemente a assimetria (7.87 → 2.59) e manteve os limites IQR estáveis — ideal para cauda longa. |
| **umidade_relativa_max**                 | **Winsorizar 5%-5%**  | Cortou outliers (23 → 0) e melhorou levemente a simetria; log/sqrt inflaram valores.                      |
| **umidade_relativa_min**                 | **Winsorizar 5%-5%**  | Assimetria e outliers foram totalmente corrigidos (1155 → 0).                                             |
| **umidade_especifica**                   | **Sqrt**              | Melhor simetria (0.89 → 0.16) e forte redução de outliers (6967 → 2102).                                  |
| **radiacao_solar**                       | **Sem transformação** | Já simétrica e sem outliers; log piorou, winsor apenas repete.                                            |
| **temperatura_min**                      | **Winsorizar 5%-5%**  | Remoção completa de outliers (5066 → 0) e leve ganho de simetria.                                         |
| **temperatura_max**                      | **Winsorizar 5%-5%**  | Mesmo comportamento de `temperatura_min`.                                                                 |
| **velocidade_vento**                     | **Log(x + 1)**        | Reduziu assimetria (1.23 → 0.19) e outliers (8723 → 1128) sem eliminar extremos reais.                    |
| **indice_queima**                        | **Winsorizar 5%-5%**  | Log e sqrt aumentaram outliers via IQR; winsor eliminou-os com mínima distorção.                          |
| **umidade_combustivel_morto_100_horas**  | **Sqrt**              | Forte queda de outliers (44 → 9) e leve suavização de forma.                                              |
| **umidade_combustivel_morto_1000_horas** | **Sqrt**              | Reduziu outliers (373 → 4) mantendo distribuição coerente.                                                |
| **componente_energia_lancada**           | **Sem transformação** | Já equilibrada; transformações criam falsos outliers.                                                     |
| **evapotranspiracao_real**               | **Sqrt**              | Melhorou drasticamente a simetria (0.71 → -0.00) e reduziu outliers (3292 → 153).                         |
| **evapotranspiracao_potencial**          | **Log(x + 1)**        | Skew caiu (0.53 → -0.36) e outliers zeraram (679 → 0).                                                    |
| **deficit_pressao_vapor**                | **Log(x + 1)**        | Alta cauda direita suavizada (1.46 → 0.52) e outliers despencaram (14758 → 672).                          |


In [ ]:
# @title Classe responsável pelo tratamento dos outliers

class OutliersTreatment(BaseEstimator, TransformerMixin):

    def __init__(self, 
                 # Colunas a serem aplicadas as transformações
                 log_columns=None, sqrt_columns=None, winsor_columns=None, 

                 # Frações dos dados a serem cortadas em cada extremidade
                 winsor_limits=(0.05, 0.05)):
        
        self.log_columns = log_columns or []
        self.sqrt_columns = sqrt_columns or []
        self.winsor_columns = winsor_columns or []
        self.winsor_limits = winsor_limits

    # Calcula parâmetros necessários para aplicar as transformações corretamente
    def fit(self, dataframe, target=None):

        # Garante que o usuário esteja enviado um dataframe no formato correto
        dataframe = dataframe if isinstance(dataframe, pd.DataFrame) else pd.DataFrame(dataframe)

        # Cálculo de offsets para garantir que não haverão valores zerados ou negativos
        # "coerce" convete valores inválidos para NaN

        self.log_offset_ = {}
        for column in self.log_columns:
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                min_value = series.min()
                self.log_offset_[column] = (abs(min_value) + 1) if pd.notna(min_value) and min_value <= 0 else 1.0

        self.sqrt_offset_ = {}
        for column in self.sqrt_columns:
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                min_value = series.min()
                self.sqrt_offset_[column] = (abs(min_value) + 0.01) if pd.notna(min_value) and min_value < 0 else 0.0

        # Garante que a winsorização só seja feita com colunas que realmente estão no dataframe
        actual_columns_to_winsor = [column for column in self.winsor_columns if column in dataframe.columns]
        low_quantile, high_quantile = self.winsor_limits

        if actual_columns_to_winsor:
            # Converte cada coluna para numérico (coerces -> NaN) e calcula quantis por coluna
            df_w = dataframe[actual_columns_to_winsor].apply(pd.to_numeric, errors="coerce")
            self.low_  = df_w.quantile(low_quantile)
            self.high_ = df_w.quantile(1 - high_quantile)
        else:
            # garante atributos vazios para não quebrar no transform()
            self.low_  = pd.Series(dtype=float)
            self.high_ = pd.Series(dtype=float)

        return self

    # Aplica as transformações
    def transform(self, dataframe):

        # Garante que o usuário esteja enviado o dataframe correto
        dataframe = dataframe.copy() if isinstance(dataframe, pd.DataFrame) else pd.DataFrame(dataframe).copy()

        # LOG
        for column, offset in self.log_offset_.items():
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                dataframe[column] = np.log(series + offset)

        # SQRT
        for column, offset in self.sqrt_offset_.items():
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                dataframe[column] = np.sqrt(series + offset)

        # Winsorização
        for column in self.low_.index:
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                dataframe[column] = series.clip(lower=self.low_[column], upper=self.high_[column])

        return dataframe

## 3. Normalização dos dados

TO-DO: trazer o algoritmo desenvolvido para normalização dos dados

In [ ]:
# @title Algoritmo de normalização

# Colunas categóricas criadas no passo de features 
_categoric_cols = ['estacao_ano_num', 'regiao_incendio'] 

def _numeric_columns_selector(dataframe): 
    # Seleciona colunas numéricas, exceto as categóricas codificadas numericamente
    num = dataframe.select_dtypes(include='number').columns.tolist() 
    return [column for column in num if column not in _categoric_cols] 

_numeric_columns_pipe = Pipeline(steps=[ 
    ('imputer', SimpleImputer(strategy='median')), 
    ('scaler', StandardScaler()), 
    ]) 

_categoric_columns_pipe = Pipeline(steps=[ 
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False)), 
])

DataNormalization = ColumnTransformer(
    transformers=[
        ('num', _numeric_columns_pipe, _numeric_columns_selector),
        ('cat', _categoric_columns_pipe, _categoric_cols)
    ],
    remainder='passthrough'  # mantém quaisquer colunas não listadas
)

## 4. Pipeline

In [ ]:
# @title Código da Pipeline

log_columns = [
    "precipitacao",
    "velocidade_vento",
    "evapotranspiracao_potencial",
    "deficit_pressao_vapor",
]
sqrt_columns = [
    "umidade_especifica",
    "umidade_combustivel_morto_100_horas",
    "umidade_combustivel_morto_1000_horas",
    "evapotranspiracao_real",
]
winsor_columns = [
    "umidade_relativa_max",
    "umidade_relativa_min",
    "temperatura_min",
    "temperatura_max",
    "indice_queima",
]

preprocess = Pipeline(steps=[
    ("features_creation", FeaturesCreation()),
    ("outliers_treatment", OutliersTreatment(
        log_cols=log_columns,
        sqrt_cols=sqrt_columns,
        winsor_cols=winsor_columns
    )),
    ("data_normalization", DataNormalization)
])


## 6. Separação em treino/teste

### Time Series Cross Validation

A Time Series Cross Validation é uma técnica especializada para validar modelos quando os dados possuem ordem cronológica. Diferente das técnicas tradicionais, ela respeita a estrutura temporal dos dados.

In [ ]:
# @title Algoritmo de separação

# Gera folds (train_idx, test_idx) respeitando ordem temporal por grupo, embargo em nível de grupo e exclusão mútua treino/teste por grupo.
def group_time_series_cross_validation(
    dataframe: pd.DataFrame,
    time_column: str,
    group_column: str,
    *,
    folds_amount: int = 5,
    fold_groups_size: int = 1,
    gap_between_groups_amount: int = 0,
    is_train_expanding_by_each_split: bool = True,
    min_train_groups: int | None = None,
    groups_amount_by_step: int | None = None,
):
    # Ordena grupos pelo primeiro timestamp
    first_time = (
        dataframe[[group_column, time_column]]
        .dropna(subset=[time_column])
        .groupby(group_column)[time_column]
        .min()
        .sort_values()
    )
    ordered_groups = first_time.index.to_numpy()
    ordered_groups_len = len(ordered_groups)

    if groups_amount_by_step is None:
        groups_amount_by_step = fold_groups_size

    if min_train_groups is None:
        min_train_groups = max(1, fold_groups_size) # pelo menos 1

    # Âncora: último grupo incluso no treino
    # Precisamos garantir espaço para gap + teste à frente
    max_anchor = ordered_groups_len - gap_between_groups_amount - fold_groups_size
    if max_anchor <= min_train_groups:
        return  # não há splits possíveis

    splits = 0
    anchor = min_train_groups
    while anchor <= max_anchor and splits < folds_amount:
        if is_train_expanding_by_each_split:
            train_groups = ordered_groups[:anchor]
        else:
            start = max(0, anchor - min_train_groups)
            train_groups = ordered_groups[start:anchor]

        test_start = anchor + gap_between_groups_amount
        test_end = test_start + fold_groups_size
        test_groups = ordered_groups[test_start:test_end]

        train_idx = dataframe.index[dataframe[group_column].isin(train_groups)].to_numpy()
        test_idx  = dataframe.index[dataframe[group_column].isin(test_groups)].to_numpy()

        if train_idx.size and test_idx.size:
            yield (train_idx, test_idx)
            splits += 1

        anchor += groups_amount_by_step


In [ ]:
# @title Executando separação

cross_validation_splits = list(group_time_series_cross_validation(
    df=wildfires,
    time_col='data',
    group_col='fire_id',
    n_splits=5,
    test_groups_size=1,
    gap_groups=0,
    expanding=True,
))

## 7. Treinando modelo

In [ ]:
# @title Algoritmo de treinamento

# Separar features e target (mantém 'data' e 'fire_id' porque o preprocess usa essas colunas)
features = wildfires.drop(columns=["houve_incendio"])
target = wildfires["houve_incendio"].astype(int)

resultados = []

modelos = {
    "Dummy (mais frequente)": DummyClassifier(strategy="most_frequent"),
    "Regressão Logística": LogisticRegression(max_iter=1000),
    "Árvore de Decisão": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=50, n_jobs=-1),
    "Naive Bayes": GaussianNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM (linear)": SVC(kernel='linear')  # Deixe por último, ele n ta indo (apagar quando for rodar, depois vou tentar arrumar)
}

# Usar os folds temporais por grupo já definidos anteriormente
# (cv_splits veio da célula "Pipeline" com group_time_series_cv(...))
cross_validation = cross_validation_splits

# Métricas de avaliação
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0),
    'roc_auc': 'roc_auc'  # deixa o sklearn decidir entre predict_proba/decision_function
}

# Remove data e fire_id, além de converte o dataframe para o formato esperado pelos classificadores
def _sanitize_after_preprocess(features):
    # Transforma em numpy.ndarray
    if isinstance(features, pd.DataFrame):
        cols = [column for column in features.columns if column not in ('data', 'fire_id')]
        features = features[cols]
        features = features.select_dtypes(include='number')
    return features

_sanitize = FunctionTransformer(_sanitize_after_preprocess, validate=False, feature_names_out='one-to-one')

for nome, modelo in modelos.items():
    print(f"Treinando modelo: {nome}...")
    try:
        pipeline = Pipeline([
            ("preprocess", preprocess), # Executa a pipeline de pré-processamento
            ("sanitize", _sanitize), # Formata o dataframe
            ("final_imputer", SimpleImputer(strategy="constant", fill_value=0.0)), # Blindagem contra NaN
            ("clf", modelo)
        ])

        scores = cross_validate(pipeline, features, target, cv=cross_validation, scoring=scoring)

        resultados.append({
            "Modelo": nome,
            "Accuracy": np.mean(scores['test_accuracy']),
            "Precision": np.mean(scores['test_precision']),
            "Recall": np.mean(scores['test_recall']),
            "F1-score": np.mean(scores['test_f1']),
            "ROC AUC": np.mean(scores['test_roc_auc']),
        })

        print(f" Modelo {nome} treinado com sucesso.\n")

    except Exception as e:
        print(f" Erro ao rodar o modelo {nome}: {e}\n")

# Mostra os resultados
df_resultados = pd.DataFrame(resultados).sort_values("F1-score", ascending=False)
df_resultados